In [30]:
!pip install -q seaborn

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from google.colab import drive

# Recuperando dados

In [ ]:
columns = ["cotacao","P/L","P/VP","PSR", "div.yield", 
           "P/ativo","P/cap.giro","P/EBIT","P/ativ_circ.liq", "EV/EBIT", 
           "EV/EBITDA","mrg_ebit","mrg.Liq","liq.corr","ROIC", 
           "ROE","liq.2meses","patrim.liq", "div.brut/patrim","cresc.rec.5a"]

data = pd.read_csv("https://raw.githubusercontent.com/GuilhermehChaves/stock-market/main/data.csv", usecols=columns)
data.tail()

# Normalizando dados

In [33]:
scaler_x = MinMaxScaler()
data[columns[1:]] = scaler_x.fit_transform(data[columns[1:]])
data.tail()

scaler_y = MinMaxScaler()
data[['cotacao']] = scaler_y.fit_transform(data[['cotacao']])

x = data.drop('cotacao', axis = 1)
y = data.cotacao

# Separando dados de teste e previsão

In [34]:
columns_predictors = columns[1:]
cols = [tf.feature_column.numeric_column(key = c) for c in columns_predictors]
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3)

# Construindo o modelo

In [35]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(x_train.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()
model.summary()

# Treinando o modelo

In [ ]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  x_train, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

# Plotando métricas

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [cotacao]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$cotacao^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()


plot_history(history)

In [ ]:
model = build_model()

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(x_train, y_train, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(x_test, y_test, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} cotacao".format(mae))

# Realizando previsões

In [ ]:
test_predictions = model.predict(x_test).flatten()

plt.scatter(y_test, test_predictions)
plt.xlabel('True Values [cotacao]')
plt.ylabel('Predictions [cotacao]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

In [ ]:
error = test_predictions - y_test
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [cotacao]")
_ = plt.ylabel("Count")